In [1]:
import logging
from pathlib import Path
from Pegasus.api import *

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
seperate = Transformation(
                "seperate",
                site="local",
                pfn="/home/scitech/shared-data/increment.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

count = Transformation(
                "count",
                site="local",
                pfn="/home/scitech/shared-data/count.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

tar = Transformation(
                "tar",
                site="local",
                pfn="/bin/tar",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

In [4]:
tc = TransformationCatalog().add_transformations(seperate, count, tar)

In [5]:
wf = Workflow("workflow3")

In [7]:

job_seperate = Job(seperate)

fb1 = File("odd_nums.txt")
fb2 = File("even_nums.txt")
fc1 = File("odd_count.txt")
fc2 = File("even_count.txt")

job_count_odd = Job(count)\
                .add_args(fb1, fc1)\
                .add_inputs(fb1)\
                .add_outputs(fc1)

job_count_even = Job(count)\
                .add_args(fb2, fc2)\
                .add_inputs(fb2)\
                .add_outputs(fc2)

fc3 = File("result.tar.gz")

job_tar = Job(tar)\
            .add_args(fc1, fc2)\
            .add_inputs(fc1, fc2)\
            .set_stdout(fc3)



In [8]:
wf.add_jobs(job_seperate, job_count_odd, job_count_even, job_tar)
wf.add_transformation_catalog(tc)

In [9]:
try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


2020.07.08 00:46:28.665 UTC: [WARNING]  Replica Catalog is either down or connection to it was never opened  
2020.07.08 00:46:28.858 UTC: [FATAL ERROR]  
 [1] java.lang.RuntimeException: TransferEngine.java: Can't determine a location to transfer input file for lfn even_nums.txt for job count_ID0000003 at edu.isi.pegasus.planner.refiner.TransferEngine.getFilesFromRC(TransferEngine.java:1403) 

